In [165]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import random

In [166]:
batch_size = 64
block_size = 100
max_iters = 5001
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 384
n_head = 6
n_layer = 6
dropout = 0.2
max_int = 1000

In [167]:
chars = sorted(list(['\n', ' ', '+', '='] + [chr(i + 48) for i in range(10)]))
print(chars)
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

def encode(s):
    return [stoi[c] for c in s]

def decode(indices):
    return ''.join(itos[i] for i in indices)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters, device=device)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


['\n', ' ', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=']


In [168]:
def get_batch(split):
    x = torch.empty(0, dtype=torch.long, device=device)
    y = torch.empty(0, dtype=torch.long, device=device)

    for _ in range(batch_size):
        a = random.randint(0, max_int)
        b = random.randint(0, max_int)
        c = a + b

        z = f"{a}+{b}={str(c)[::-1]}"
        z = z + (block_size - len(z) - 1) * '0' + '\n'
        nx = torch.tensor([encode(z)[:-1]], dtype=torch.long, device=device)
        ny = torch.tensor([encode(z)[1:]], dtype=torch.long, device=device)

        x = torch.cat((x, nx), dim=0)
        y = torch.cat((y, ny), dim=0)

    return x, y

get_batch('train')

(tensor([[11, 10,  7,  ...,  3,  3,  3],
         [ 8,  5,  6,  ...,  3,  3,  3],
         [ 5,  6,  6,  ...,  3,  3,  3],
         ...,
         [ 6,  4,  7,  ...,  3,  3,  3],
         [11,  3,  7,  ...,  3,  3,  3],
         [ 5,  8,  7,  ...,  3,  3,  3]], device='cuda:0'),
 tensor([[10,  7,  2,  ...,  3,  3,  0],
         [ 5,  6,  2,  ...,  3,  3,  0],
         [ 6,  6,  2,  ...,  3,  3,  0],
         ...,
         [ 4,  7,  2,  ...,  3,  3,  0],
         [ 3,  7,  2,  ...,  3,  3,  0],
         [ 8,  7,  2,  ...,  3,  3,  0]], device='cuda:0'))

In [169]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        # for every head have to set all the layers? how do i organize the layers?
        #
        self.num_heads = num_heads
        self.keys = nn.ModuleList([nn.Linear(n_embed, head_size, bias=False) for _ in range(num_heads)])
        self.queries = nn.ModuleList([nn.Linear(n_embed, head_size, bias=False) for _ in range(num_heads)])
        self.values = nn.ModuleList([nn.Linear(n_embed, head_size, bias=False) for _ in range(num_heads)])
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        heads = []

        for i in range(self.num_heads):
            k = self.keys[i](x)
            q = self.queries[i](x)
            wei = q @ k.transpose(-2, -1) * C**-0.5 #how much each letter corresponds to each other one
            wei = wei.masked_fill(self.tril[:T, :T] == 0, float ('-inf'))
            wei = F.softmax(wei, dim=-1)
            wei = self.dropout(wei)
            v = self.values[i](x)
            heads.append(wei @ v)

        out = torch.cat(heads, dim=-1)
        out = self.proj(out) # Project the concatenated heads
        out = self.dropout(out) # Apply dropout after projection
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape # B is batch, T is time
        token_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = token_emb + pos_emb # (B, T, C)
        x = self.blocks(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx
    @torch.no_grad()
    def solve(self, idx):
      generated_tokens = []
      while True:
          idx_cond = idx[:, -block_size:]
          logits, loss = self(idx_cond)
          logits = logits[:, -1, :]
          probs = F.softmax(logits, dim=-1)
          idx_next = torch.multinomial(probs, num_samples=1)
          idx = torch.cat((idx, idx_next), dim = 1)
          generated_tokens.append(idx_next.item())
          if idx_next.item() == stoi['\n']:
              break

      # Decode and process the generated tokens
      generated_string = decode(generated_tokens)

      # Reverse the string
      reversed_string = generated_string[::-1]
      return reversed_string[3:].lstrip('0')

In [170]:
#train model
model = BigramLanguageModel()
m = model.to(device)
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 1.2776, val loss 1.2768
step 500: train loss 0.1898, val loss 0.1900
step 1000: train loss 0.1421, val loss 0.1423
step 1500: train loss 0.1248, val loss 0.1248
step 2000: train loss 0.1219, val loss 0.1216
step 2500: train loss 0.1197, val loss 0.1197
step 3000: train loss 0.1195, val loss 0.1193
step 3500: train loss 0.1196, val loss 0.1196
step 4000: train loss 0.1195, val loss 0.1195
step 4500: train loss 0.1193, val loss 0.1193
step 5000: train loss 0.1195, val loss 0.1194


In [171]:

a = "123"
b = "456"

context = torch.tensor([encode(f"{a}+{b}=")], device=device)
print(m.solve(context))



579
